<a href="https://colab.research.google.com/github/tvelagapudi/deblender/blob/master/Model_Assisted_GAN_Lensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout, BatchNormalization, Lambda
from keras.optimizers import Adam, RMSprop, SGD
from keras import initializers
from scipy.interpolate import interp1d

!pip install import-ipynb
import import_ipynb
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy the link and remove the front part of the link (i.e. https://drive.google.com/open?id=) to get the file ID.
your_module = drive.CreateFile({'id':'1F4Q9K86uZVRlou1ieNxohZEk5-vHZwIB'})
your_module.GetContentFile('lenstronomy1.ipynb')
import lenstronomy1 as lens

W0709 15:11:03.742194 140635728914304 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

In [0]:
class Networks(object):
    def __init__(self, noise_size=100, params=4, img_rows=28, img_cols=28, channel=1):
        self.noise_size = noise_size
        self.params = params
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.G = None # generator
        self.E = None # emulator
        self.D = None # discriminator
        self.S = None # siamese
        self.GE = None # generator + emulator
        self.SM = None # siamese model
        self.AM1 = None # adversarial model 1
        self.DM = None # discriminator model
        self.AM2 = None # adversarial model 2
    
    '''
    Generator: produce parameters such that the simulator can use them to create images that 
    cannot be distinguished from true data images
    '''
    def generator(self):
        if self.G:
            return self.G
        
        # params
        dropout = 0.0
        depth = 64
        dim = 7
        
        # input noise
        input_noise_shape = (self.noise_size,)
        input_noise_layer = Input(shape=input_noise_shape, name='input_noise')
        
        # architecture
        self.G = Dense(dim*dim*depth)(input_noise_layer)
        
        #self.G = BatchNormalization(momentum=0.9)(self.G)
        self.G = LeakyReLU(0.2)(self.G)
        self.G = Dense(dim*depth)(self.G)
        
        #self.G = BatchNormalization(momentum=0.9)(self.G)
        self.G = LeakyReLU(0.2)(self.G)
        self.G = Dropout(dropout)(self.G)
        self.G = Dense(self.params, activation='tanh')(self.G)
        
        # model
        self.G = Model(inputs=input_noise_layer, outputs=self.G, name='generator')
        
        # print
        print("Generator:")
        self.G.summary()
        return self.G

    ''' 
    Emulator: generate identical images to those of the simulator S when both E and S 
    are fed with the same input parameters
    ''' 
    def emulator(self):
        if self.E:
            return self.E

        # input params
        input_params_shape = (self.params,)
        input_params_layer = Input(shape=input_params_shape, name='input_params')

        # architecture
        self.E = Dense(1024)(input_params_layer)
        self.E = LeakyReLU(0.2)(self.E)
        self.E = Dense(7*7*128, kernel_initializer=initializers.RandomNormal(stddev=0.02))(self.E)
        self.E = LeakyReLU(0.2)(self.E)
        self.E = Reshape((7, 7, 128))(self.E)
        self.E = UpSampling2D(size=(2, 2))(self.E)
        self.E = Conv2D(64, kernel_size=(5, 5), padding='same')(self.E)
        self.E = LeakyReLU(0.2)(self.E)
        self.E = UpSampling2D(size=(2, 2))(self.E)
        self.E = Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh')(self.E)

        # model
        self.E = Model(inputs=input_params_layer, outputs=self.E, name='emulator')

        # print
        print("Emulator")
        self.E.summary()
        return self.E

    '''
    Discriminator: distinguish between true data images and images produced by the simulator (or 
    images produced by the emulator to speed up the training process)
    '''
    def discriminator(self):
        if self.D:
            return self.D

        # input image
        input_shape_image = (self.img_rows, self.img_cols, self.channel)
        input_layer_image = Input(shape=input_shape_image, name='input_image')
        
        # architecture
        self.D = Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', 
                        kernel_initializer=initializers.RandomNormal(stddev=0.02))(input_layer_image)
        self.D = LeakyReLU(0.2)(self.D)
        self.D = Dropout(0.3)(self.D)
        self.D = Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same')(self.D)
        self.D = LeakyReLU(0.2)(self.D)
        self.D = Dropout(0.3)(self.D)
        self.D = Flatten()(self.D)
        self.D = Dense(1)(self.D)
        self.D = Activation('sigmoid')(self.D)

        # model
        self.D = Model(inputs=input_layer_image, outputs=self.D, name='discriminator')

        # print
        print("Discriminator:")
        self.D.summary()
        return self.D

    '''
    Siamese: determine the similarity between images produced by the simulator and the emulator
    '''
    def siamese(self):
        if self.S:
            return self.S

        # input images
        input_shape_image = (self.img_rows, self.img_cols, self.channel)
        input_image_anchor = Input(shape=input_shape_image, name='input_image_anchor')
        input_image_candid = Input(shape=input_shape_image, name='input_image_candidate')
        input_image = Input(shape=input_shape_image, name='input_image')

        # siamese
        cnn = Conv2D(64, kernel_size=(8, 8), strides=(2, 2), padding='same', 
                     kernel_initializer=initializers.RandomNormal(stddev=0.02))(input_image)
        cnn = LeakyReLU(0.2)(cnn)
        #cnn = Dropout(0.3)(cnn)
        cnn = Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same')(cnn)
        cnn = LeakyReLU(0.2)(cnn)
        #cnn = Dropout(0.3)(cnn)
        cnn = Flatten()(cnn)
        cnn = Dense(128, activation='sigmoid')(cnn)
        cnn = Model(inputs=input_image, outputs=cnn, name='cnn')

        # left and right encodings         
        encoded_l = cnn(input_image_anchor)
        encoded_r = cnn(input_image_candid)

        # merge two encoded inputs with the L1 or L2 distance between them
        L1_distance = lambda x: K.abs(x[0]-x[1])
        L2_distance = lambda x: (x[0]-x[1]+K.epsilon())**2/(x[0]+x[1]+K.epsilon())
        both = Lambda(L2_distance)([encoded_l, encoded_r])
        prediction = Dense(1, activation='sigmoid')(both)

        # model
        self.S = Model([input_image_anchor, input_image_candid], output=prediction, name='siamese')

        # print
        print("Siamese:")
        self.S.summary()
        return self.S

    '''
    Generator + emulator: output emulated images from input noise
    '''

    def generator_emulator(self):
        if self.GE:
            return self.GE

        # input noise
        input_noise_shape = (self.noise_size,)
        input_noise_layer = Input(shape=input_noise_shape, name='input_noise') 

        # generator
        generator_ref = self.generator()
        generator_ref.trainable = False
        self.GE = generator_ref(input_noise_layer)

        # emulator
        emulator_ref = self.emulator()
        emulator_ref.trainable = False
        self.GE = emulator_ref(self.GE)

        # model
        self.GE = Model(inputs=input_noise_layer, outputs=self.GE, name='generator_emulator')

        # print
        print("Generator + Emulator:")
        self.GE.summary()
        return self.GE

    '''
    Discriminator model
    '''
    def discriminator_model(self):
        if self.DM:
            return self.DM
          
        # optimizer
        #optimizer = SGD(lr=0.0001)
        optimizer = Adam(lr=0.0001)

        # input image
        input_image_shape = (self.img_rows, self.img_cols, self.channel)
        input_image_layer = Input(shape=input_image_shape, name='input_image')

        # discriminator
        discriminator_ref = self.discriminator()
        discriminator_ref.trainable = True
        self.DM = discriminator_ref(input_image_layer)

        # model
        self.DM = Model(inputs=input_image_layer, outputs=self.DM, name='discriminator_model')
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        print("Discriminator model") 
        self.DM.summary()
        return self.DM

    '''
    Siamese model
    '''
    def siamese_model(self):
        if self.SM:
            return self.SM

        # optimizer
        #optimizer = SGD(lr=0.0001)
        optimizer = Adam(lr=0.0001)

        # input images
        input_shape_image = (self.img_rows, self.img_cols, self.channel)
        input_image_anchor = Input(shape=input_shape_image, name='input_image_anchor')
        input_image_candid = Input(shape=input_shape_image, name='input_image_candidate')
        input_layer = [input_image_anchor, input_image_candid]

        # discriminator
        siamese_ref = self.siamese()
        siamese_ref.trainable = True
        self.SM = siamese_ref(input_layer)

        # model
        self.SM = Model(inputs=input_layer, outputs=self.SM, name='siamese_model')
        self.SM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        
        #print
        print("Siamese model") 
        self.SM.summary()
        return self.SM

    '''
    Adversarial 1 model
    '''
    def adversarial1_model(self):
        if self.AM1:
            return self.AM1
         
        #optimizer
        optimizer = Adam(lr=0.0001)

        # input 1: simulated image
        input_image_shape = (self.img_rows, self.img_cols, self.channel)
        input_image_layer = Input(shape=input_image_shape, name='input_image')

        # input 2: params
        input_params_shape = (self.params,)
        input_params_layer = Input(shape=input_params_shape, name='input_params')

        # emulator
        emulator_ref = self.emulator()
        emulator_ref.trainable = True
        self.AM1 = emulator_ref(input_params_layer)

        # siamese
        siamese_ref = self.siamese()
        siamese_ref.trainable = False
        self.AM1 = siamese_ref([input_image_layer, self.AM1])

        # model
        input_layer = [input_image_layer, input_params_layer] 
        self.AM1 = Model(inputs=input_layer, outputs=self.AM1, name='adversarial_1_model')
        self.AM1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # print
        print("Adversarial 1 model:")
        self.AM1.summary()
        return self.AM1

    '''
    Adversarial 2 model
    '''

    def adversarial2_model(self):
        if self.AM2:
            return self.AM2
          
        #optimizer
        optimizer = Adam(lr=0.0001)

        # input noise
        input_noise_shape = (self.noise_size,)
        input_noise_layer = Input(shape=input_noise_shape, name='input_noise')

        # generator
        generator_ref = self.generator()
        generator_ref.trainable = True
        self.AM2 = generator_ref(input_noise_layer)

        # emulator
        emulator_ref = self.emulator(
        emulator_ref.trainable = False
        self.AM2 = emulator_ref(self.AM2)

        # discriminator
        discriminator_ref = self.discriminator()
        discriminator_ref.trainable = False
        self.AM2 = discriminator_ref(self.AM2)

        # model
        self.AM2 = Model(inputs=input_noise_layer, outputs=self.AM2, name='adversarial_2_model')
        self.AM2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # print
        print("Adversarial 2 model")
        self.AM2.summary()
        return self.AM2


# manually specify the GPUs to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# remove for randomness
np.random.seed(7)
net_range  = [-1,1]

# Case study 1 ranges and mapping
m_range = [-2,2]
b_range = [-2,2]
x_range = [0,12]
l_range = [5,15]
m_m = interp1d(net_range,m_range)
m_b = interp1d(net_range,b_range)
m_x = interp1d(net_range,x_range)
m_l = interp1d(net_range,l_range)

# Case study 2 ranges and mapping
x0_range = [5,23]
y0_range = [5,23]
r_range  = [2,14]
n_range  = [0,0.5]
b_range  = [0.5,1]
m_x0 = interp1d(net_range,x0_range)
m_y0 = interp1d(net_range,y0_range)
m_r  = interp1d(net_range,r_range)
m_n  = interp1d(net_range,n_range)
m_b  = interp1d(net_range,b_range)

class ModelAssistedGAN(object):
    # Output parameters for true data
    def data_params(self, cs='cs1'):
        
        # Case study 1
        if cs=='cs1':
            while(True):
                m, b, x0, length = [np.random.normal(1.5, 0.3, 1)[0], np.random.normal(0.5, 0.1, 1)[0], 
                                    np.random.normal(10, 0.5, 1)[0], np.random.normal(9, 0.5, 1)[0]]
                y = int(round(m*x0+b))
                if y >= -(self.img_rows//2) and y<(self.img_rows//2):
                    break
            return m, b, x0, length

        # Case study 2
        x0, y0, r, n, b = [np.random.normal(18, 0.8, 1)[0], np.random.normal(18, 0.8, 1)[0],
                           np.random.normal(8, 0.5, 1)[0], np.random.normal(0.15, 0.05, 1)[0], 
                           np.random.normal(0.9, 0.05, 1)[0]]
        return x0, y0, r, n, b
    
    # Fill pixel map for case study 1 
    """def fillPixelMapCS1(self, m, b, x0, length, pixelMap):
        max_len = pixelMap.shape[0]
        value = 1.0
        x0 = int(round(float(x0)))
        length = int(round(float(length))) 
        for x in range(x0, x0+length):
            if x >= max_len or x < 0:
                continue
            y=int(round(m*x+b))+max_len//2
            if y >= max_len or y < 0:
                continue
            pixelMap[x,y,0]=value

    # Fill pixel map for case study 2
    def fillPixelMapCS2(self, x0, y0, r, n, b, pixelMap):
        def putpixel(self, x, y, b, pixelMap):
            if x>=0 and y>=0 and x<pixelMap.shape[0] and y<pixelMap.shape[1]:
                pixelMap[x,y,0] = max(min(np.random.normal((2*b)-1, 0.05, 1)[0], 1.0), -1.0)
        x0 = int(round(x0))
        y0 = int(round(y0))
        r  = int(round(r))
        x = r-1
        y = 0
        dx = 1
        dy = 1
        err = dx - (r << 1)
        
        # noise
        for row in range(pixelMap.shape[0]):
            for col in range(pixelMap.shape[1]):
                pixelMap[row, col, 0] = max(min(np.random.normal((2*n)-1, 0.05, 1)[0], 1.0), -1.0)

        # signal
        while (x >= y):
            putpixel(x0 + x, y0 + y, b, pixelMap)
            putpixel(x0 + y, y0 + x, b, pixelMap)
            putpixel(x0 - y, y0 + x, b, pixelMap)
            putpixel(x0 - x, y0 + y, b, pixelMap)
            putpixel(x0 - x, y0 - y, b, pixelMap)
            putpixel(x0 - y, y0 - x, b, pixelMap)
            putpixel(x0 + y, y0 - x, b, pixelMap)
            putpixel(x0 + x, y0 - y, b, pixelMap)
            if (err <= 0):
                y+=1
                err += dy
                dy += 2
            if (err > 0):
                x-=1
                dx += 2
                err += dx - (r << 1)"""

    def __init__(self, noise_size=100, params=4, img_rows=28, img_cols=28, channel=1):
        self.noise_size = noise_size
        self.params = params
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.Networks = Networks(noise_size=noise_size, params=params, img_rows=img_rows, 
                        img_cols=img_cols, channel=channel)
        self.discriminator = self.Networks.discriminator_model()
        self.siamese = self.Networks.siamese_model()
        self.adversarial1 = self.Networks.adversarial1_model()
        self.adversarial2 = self.Networks.adversarial2_model()
        self.generator = self.Networks.generator()
        self.generator_emulator = self.Networks.generator_emulator()
        self.emulator = self.Networks.emulator()

    def train(self, train_steps=100000, batch_size=128):
        '''
        Pre-training stage
        '''
        for train_step in range(train_steps):
            log_mesg = '%d:' % train_step
            noise_value = 0.05
            params_list = np.random.uniform(-1.0, 1.0, size=[batch_size, self.params])
            y_ones = np.ones([batch_size, 1])
            y_zeros = np.zeros([batch_size, 1])

            '''
            Step 1
            '''
            
            # simulated images
            images_simu = np.zeros((batch_size, self.img_rows,self.img_cols,self.channel))
            images_simu.fill(-1)
            # commented lines (case study 2)
            #images_simu2 = np.zeros((batch_size, self.img_rows,self.img_cols,self.channel)) # case study 2
            #images_simu2.fill(-1) # case study 2
            for index in range(batch_size):
                m, b, x0, length = params_list[index]
                self.fillPixelMapCS1(m_m(m), m_b(b), m_x(x0), m_l(length), images_simu[index])
                # commented lines (case study 2)
                #x0, y0, r, n, b = params_list[index]
                #self.fillPixelMapCS2(m_x0(x0), m_y0(y0), m_r(r), m_n(n), m_b(b), images_simu[index])
                #self.fillPixelMapCS2(m_x0(x0), m_y0(y0), m_r(r), m_n(n), m_b(b), images_simu2[index])
            images_simu_copy = np.copy(images_simu)
            # commented lines (case study 2)
            #images_simu2_copy = np.copy(images_simu2)

            # emulated images
            images_emul = self.emulator.predict(params_list)
            images_emul_copy = np.copy(images_emul)

            # noise
            y_ones = np.array([np.random.uniform(0.7,1.0) for x in range(batch_size)]).reshape([batch_size, 1])
            y_zeros = np.array([np.random.uniform(0,0.3) for x in range(batch_size)]).reshape([batch_size, 1])
            if noise_value > 0:
                for index in range(batch_size):
                    if np.random.random() < noise_value:
                        images_simu_copy[index], images_emul_copy[index] = images_emul[index], images_simu[index]


            # train siamese
            d_loss_simu = self.siamese.train_on_batch([images_simu_copy, images_simu_copy], y_ones)
            # commented lines (case study 2)
            #d_loss_simu = self.siamese.train_on_batch([images_simu_copy, images_simu2_copy], y_ons)
            d_loss_fake = self.siamese.train_on_batch([images_simu_copy, images_emul_copy], y_zeros)
            d_loss = 0.5 * np.add(d_loss_simu, d_loss_fake)
            log_mesg = "%s [S loss: %f]" % (log_mesg, d_loss[0])

            '''
            Step 2
            '''

            #for index in range(batch_size):
            #    if np.random.random() < noise_value:
            #        y_ones[index, 0] = np.random.uniform(0,0.3)
            y_ones = np.ones([batch_size, 1])

            # train emulator
            a_loss = self.adversarial1.train_on_batch([images_simu, params_list], y_ones)
            log_mesg = "%s [E loss: %f]" % (log_mesg, a_loss[0])
            print(log_mesg)

        '''
        Training stage
        '''
        for train_step in range(train_steps):
            log_mesg = '%d:' % train_step
            noise_value = 0.3
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, self.noise_size])
            params_list = self.generator.predict(noise)
            y_ones = np.ones([batch_size, 1])
            y_zeros = np.zeros([batch_size, 1])
            
            '''
            Step 1
            '''
            
            # true images
            images_true = np.zeros((batch_size, self.img_rows, self.img_cols, self.channel))
            images_true.fill(-1)       
            for index in range(batch_size):
                m, b, x0, length = self.data_params()
                self.fillPixelMapCS1(m, b, x0, length, images_true[index])
                # commented lines (case study 2)
                #x0, y0, r, n, b = params_list[index]
                #self.fillPixelMapCS2(m_x0(x0), m_y0(y0), m_r(r), m_n(n), m_b(b), images_true[index])
            images_true_copy = np.copy(images_true) 

            # simulated images
            images_simu = np.zeros((batch_size, self.img_rows,self.img_cols, self.channel))
            images_simu.fill(-1)
            for index in range(batch_size):
                m, b, x0, length = params_list[index]
                self.fillPixelMapCS1(m_m(m), m_b(b), m_x(x0), m_l(length), images_simu[index])
                # commented lines (case study 2)
                #x0, y0, r, n, b = params_list[index]
                #self.fillPixelMapCS2(m_x0(x0), m_y0(y0), m_r(r), m_n(n), m_b(b), images_simu[index])
            images_simu_copy = np.copy(images_simu)

            # noise
            y_ones = np.array([np.random.uniform(0.7,1.2) for x in range(batch_size)]).reshape([batch_size, 1])
            y_zeros = np.array([np.random.uniform(0,0.3) for x in range(batch_size)]).reshape([batch_size, 1])
            if np.random.random() < noise_value:
                for index in range(batch_size):
                    if np.random.random() < noise_value:
                        images_true_copy[index], images_simu_copy[index] = images_simu[index], images_true[index]

            # train discriminator
            d_loss_true = self.discriminator.train_on_batch(images_true_copy, y_ones)
            d_loss_simu = self.discriminator.train_on_batch(images_simu_copy, y_zeros)
            d_loss = 0.5 * np.add(d_loss_true, d_loss_simu)
            log_mesg = "%s [D loss: %f]" % (log_mesg, d_loss[0])

            '''
            Step 2
            '''

            # noise
            for index in range(batch_size):
                if np.random.random() < noise_value:
                    y_ones[index, 0] = np.random.uniform(0,0.3)

            # train generator
            a_loss = self.adversarial2.train_on_batch(noise, y_ones)
            log_mesg = "%s [G loss: %f]" % (log_mesg, a_loss[0])
            print(log_mesg)
            noise_value*=0.999

if __name__ == '__main__':
    noise_size=100
    params=4
    img_rows=28
    img_cols=28
    channel=1
    magan = ModelAssistedGAN(noise_size=noise_size, params=params, img_rows=img_rows, img_cols=img_cols,
                             channel=channel)
    magan.train(train_steps=500000, batch_size=128)